<a href="https://colab.research.google.com/github/eswens13/deep_learning/blob/master/keras/mnist_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MNIST Classifier

Exploring an introductory deep learning problem to solidify some of the concepts I've learned thusfar.

In [1]:
from keras.datasets import mnist
from keras.utils import to_categorical

(X_train, y_train), (X_test, y_test) = mnist.load_data()
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
X_train = X_train[:,:,:,None]
X_test = X_test[:,:,:,None]
print("Data Shapes:\n\tX_train: {}\n\ty_train: {}\n\tX_test: {}\n\ty_test{}"\
      .format(X_train.shape, y_train.shape, X_test.shape, y_test.shape))

Using TensorFlow backend.


11493376/11490434 [==============================] - 0s 0us/step
Data Shapes:
	X_train: (60000, 28, 28, 1)
	y_train: (60000, 10)
	X_test: (10000, 28, 28, 1)
	y_test(10000, 10)


In [2]:
import os

# Get a tool called ngrok to use as a tunnel between my local machine and the
# Google Colab server. This will allow us to use TensorBoard to visualize
# helpful metrics of the network.
#
# Tutorial:
#   https://www.dlology.com/blog/quick-guide-to-run-tensorboard-in-google-colab/
#
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

# Now the ngrok exectuable is extracted to the current directory. Check to make
# sure there is a log directory for Keras to use.
cwd = os.getcwd()
LOG_DIR = os.path.join(cwd, 'log')
print("Log Dir: {}".format(LOG_DIR))
if not os.path.exists(LOG_DIR):
  os.system('mkdir -p {}'.format(LOG_DIR))

# Run tensorboard in the background.
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

# Tell ngrok (in the background) to tunnel TensorBoard port 6006 to the outside
# world.
get_ipython().system_raw('./ngrok http 6006 &')

# Get the URL that I can use to hook into TensorBoard from my local machine.
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2019-05-27 23:07:27--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 35.173.6.94, 52.55.191.55, 34.226.180.131, ...
Connecting to bin.equinox.io (bin.equinox.io)|35.173.6.94|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16648024 (16M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  15.88M  78.2MB/s    in 0.2s    

2019-05-27 23:07:27 (78.2 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [16648024/16648024]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Log Dir: /content/log
http://fc7d94b3.ngrok.io


In [0]:
from keras.models import Sequential
from keras.layers import Activation, Conv2D, Dense, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.regularizers import l2

import numpy as np

# Build the Keras model
def get_keras_model():
  
  model = Sequential();
  num_filters = 16
  kernel_size = [3,3]
  stride_size = [1,1]
  pad_str = 'same'
  format_str = 'channels_last'
  act_str = 'relu'
  
  model.add(Conv2D(num_filters, \
                   kernel_size, \
                   strides=stride_size, \
                   padding=pad_str, \
                   data_format=format_str, \
                   use_bias=True, \
                   activation=act_str, \
                   kernel_regularizer=l2(0.01)))
  
  pool_window = [2,2]
  stride_size = [2,2]
  pad_str = 'valid'
  model.add(MaxPooling2D(pool_size=pool_window, \
                         strides=stride_size, \
                         padding=pad_str, \
                         data_format=format_str))

  num_filters = 32
  model.add(Conv2D(num_filters, \
                   kernel_size, \
                   strides=stride_size, \
                   padding=pad_str, \
                   data_format=format_str, \
                   use_bias=True, \
                   activation=act_str, \
                   kernel_regularizer=l2(0.01)))
  
  model.add(MaxPooling2D(pool_size=pool_window, \
                         strides=stride_size, \
                         padding=pad_str, \
                         data_format=format_str))
  
  '''num_filters = 256
  model.add(Conv2D(num_filters, \
                   kernel_size, \
                   strides=stride_size, \
                   padding=pad_str, \
                   data_format=format_str, \
                   use_bias=True, \
                   activation=act_str, \
                   kernel_regularizer=l2(0.01)))
  
  pad_str = 'valid'
  model.add(MaxPooling2D(pool_size=pool_window, \
                         strides=stride_size, \
                         padding=pad_str, \
                         data_format=format_str))'''
  
  model.add(Flatten(data_format=format_str))
  
  model.add(Dense(512, activation=act_str, use_bias=True))
  model.add(Dense(128, activation=act_str, use_bias=True))
  model.add(Dense(64, activation=act_str, use_bias=True))
  
  # Output Layer
  model.add(Dense(10, activation=act_str, use_bias=True))
  
  # Compile the model.
  adam = Adam(lr=1e-4)
  model.compile(loss='categorical_crossentropy', \
                optimizer=adam, \
                metrics=['accuracy'])
  
  return model

In [0]:
# Create a Keras callback so that it outputs to TensorBoard rather than this
# console.
from keras.callbacks import TensorBoard

tbCallback = TensorBoard(log_dir=LOG_DIR, \
                         histogram_freq=1, \
                         write_graph=False, \
                         write_grads=True, \
                         batch_size=100, \
                         write_images=False)

In [5]:
model = get_keras_model()
model.fit(X_train, y_train, \
          epochs=5, batch_size=100, verbose=2, callbacks=[tbCallback], \
          validation_data=(X_test, y_test))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
 - 7s - loss: 2.5797 - acc: 0.6538 - val_loss: 2.2037 - val_acc: 0.4403
Epoch 2/5
 - 3s - loss: 1.0452 - acc: 0.8353 - val_loss: 0.9795 - val_acc: 0.8181
Epoch 3/5
 - 3s - loss: 0.9383 - acc: 0.8343 - val_loss: 0.7729 - val_acc: 0.9159
Epoch 4/5
 - 3s - loss: 1.0266 - acc: 0.7786 - val_loss: 0.8250 - val_acc: 0.8840
Epoch 5/5
 - 3s - loss: 1.0596 - acc: 0.7538 - val_loss: 0.6849 - val_acc: 0.9215


In [8]:
scores = model.evaluate(X_test, y_test)
print("{}: {}\n{}: {}".format( \
      model.metrics_names[0], scores[0], model.metrics_names[1], scores[1]))

10000/10000 [==============================] - 1s 55us/step
loss: 0.6848655523777009
acc: 0.9215
